# Kernel 3: Shared Memory Cache-Blocking

We have mentioned, quite often actually, how threads in a block have access to shared memory, but we have not really taken advantage of this yet. 

In the previous kernel, even though we optimized our memory accesses by coalescing, the memory accesses themselves are still not optimised since they are made to global memory.

If you recall, SMs hold blocks, and is where the hardware execution actually occurs (on warps in those blocks). Each SM has one shared memory (SMEM), physically located on the chip, that is partitioned among the chunks. This allows threads within a block to communicate with the other threads in the block using the shared memory chunk.

A small visualization of this (of a A100 GPU in particular):

![](../../images/GEMM1/smem.png)

A cool thing about this is how it is configurable. We can trade off a larger shared memory for each SM for a smaller L1 cache.

Since the shared memory is on-chip, it has a much lower latency and bandwidth.

In a nutshell, our approach here is to utilise this shared memory, by loading in chunks of both A and B from global memory into shared memory, then performing as much work as possible, before moving onto the next chunks. If you are seeing what's happening (no worries, if not) then you will see that we are performing partial sums on C until the result is computed.

We are essentially computing a whole chunk of C, by using chunks of both the corresponding rows in A and corresponding columns in B.

A visualization:

![](../../images/GEMM1/smemchunks.png)

If we look at the 32x32 chunk in C, it needs all the 32 rows in A (that chunk A covers) and needs all 32 columns in B (that chunk B covers), so instead of doing the one row * one column at time to compute one complete entry, we take a chunk of the row(s) and column(s) to compute partial entries for a larger amouunt of threads. 

This way we just need to use these chunks of A and B, sliding along, and never having to reuse elements for the chunk of C we are computing. Once a chunk has been loaded in (from both A and B) all the computation it is needed for (partial dot products) will be done.

The main power in this lies in the fact that these chunks are in shared memory, and used by all the threads that are involved in computing the output chunk in C.

To really hammer down why this optimises memory accesses, look at this small example:


### Code
If some parts of this optimization still do not make sense, let's step through the code and it might illuminate some confusion points

In [ ]:
// advance pointers to the starting positions
A += cRow * BLOCKSIZE * K;                    // row=cRow, col=0
B += cCol * BLOCKSIZE;                        // row=0, col=cCol
C += cRow * BLOCKSIZE * N + cCol * BLOCKSIZE; // row=cRow, col=cCol

Pointer to A skips past the first #cRow number of chunks (each of BLOCKSIZE) which is why we also multiply by K, pointing to the start of the chunk in question. Similarly, for the pointer to B, we skip past the first #cCol number of cols, pointing to the start of the chunk in question.
Finally, for the pointer to C, a combination of the two is done, skipping past cRow number of rows, and then past cCol number of columns to reach the exact chunk in C that is being computed.

In [ ]:
__shared__ float As[BLOCKSIZE][BLOCKSIZE];
__shared__ float Bs[BLOCKSIZE][BLOCKSIZE];

This is the declaration of two arrays that will be allocated in the on-chip shared memory - visible to all threads in the block.

One misconception that I had, that you might share, is that since kernels are ran from the perspective of a single, individual thread, I thought 'Hey, wouldn't this mean we are redeclaring this shared memory array for every thread, even ones in the same block?'. 

Well, no, as the CUDA compiler, upon seeing __shared__ allocates only one copy per block in SMEM. Good intuition, though!

In [ ]:
float tmp = 0.0;

// the outer loop advances A along the columns and B along
// the rows until we have fully calculated the result in C.
for (int bkIdx = 0; bkIdx < K; bkIdx += BLOCKSIZE) {
        // ... outer loop body
    }

This outer loop body advances along the columns of A and the rows of B until we have fully calculated the result in C

Taking a look inside the outer loop:

In [ ]:
  As[threadIdx.y * BLOCKSIZE + threadIdx.x] = A[threadIdx.y * K + threadIdx.x];
  Bs[threadIdx.y * BLOCKSIZE + threadIdx.x] = B[threadIdx.y * N + threadIdx.x];
  
  __syncthreads();


We can then have each thread load one of the elements in A & B from global memory to shared memory, inserting it into our shared memory arrays that we defined earlier. 

A thing to wonder is how can we simply use the local threadIdx i.e. the row and column of the thread in their block to index the matrices A and B - this is since we moved the pointers for A & B to skip to the chunk in question,  being at the correct row/coolume origin point.

Since we are essentially in the chunks of matrix A & B that are being used by the current block, we can use local thread addressing, and since we are using the threadIdx.x to index the threads, global memory coalescing is back into play!

Then further, since the shared memory arrays we created, As & Bs are also specific to this current block, they can also be addressesed with the same local IDs.

__syncthreads() blocks the execution until all the threads in the block have reached this point, i.e. the chunks from A & B are completely loaded in. This ensures that the whole tile is in SMEM before any thread uses it to perform dot products. 

Also one thing that has to be understood is that since threads in a warp execute the same instruction together, a 'early thread' would actually mean an early warp i.e. the whole warp has reached the __syncthreads() point and needs to wait for the other warps in the block to also reach the barrier. Once all warps in the block have stalled at that barrier, the barrier is released and all warps can continue.

In [ ]:
// inner loop
  for (int dotIdx = 0; dotIdx < BLOCKSIZE; ++dotIdx) {
    tmp += As[threadIdx.y * BLOCKSIZE + dotIdx] *
            Bs[dotIdx * BLOCKSIZE + threadIdx.x];
  }

  __syncthreads();

This inner loop now computes the partial dot product for this first chunk - all of which is cached in SMEM.

Again, pretty standard indexing, we just skip to the exact row and exact column in the cached chunks of the current thread and just iterate through it for the whole chunk. 

It is evident, but just to clarify, as I had a small moment of confusion with this, look below at the example:

![](../../images/GEMM1/smemdotproduct.png)

The singular thread indicated by the red dot in C (squint to see it!) needs the entire row in A and entire column in B, denoted by red lines, to compute its entry in C. This is why, the kernel, even though it is from the perspective of a single thread, will use all the entries in the cached chunks As & Bs.

This is the power of using SMEM, as our thread only loaded in a single entry from A & B to put into As & Bs but now can use the whole row and whole column of elements that were loaded in by other threads in the block - sharing is caring!

So, again, just to consolidate the information: As & Bs act as shared memory buffers for the whole block, so all threads first cooperatively fill them up, and then in the inner loop, they all consume the same populated As & Bs.

A caveat, however, is that threads (warps) don't all run at the exact same pace and some might finish their 32 multiplies quickly while others might get delayed.

This is why we have another __syncthreads() call as this ensures that the early finishing threads (warps) don't race ahead to the next outer iteration and then overwrite As with the new chunk, corrupting it for the threads (warps) catching up, currently in the inner loop.

In [ ]:
  // advance pointers onto next chunk
  A += BLOCKSIZE;
  B += BLOCKSIZE * N;

Now, we simply advance the pointers to A and B onto the next chunk. Again, take a moment to look back at the visualisations and this will be a lot more clear.

And with this, the outer loop ends.

Just one more affair to deal with: again, stop here for a second and try to think of what is remaining, and then check your answer below.

In [ ]:
C[threadRow * N + threadCol] =
    alpha * tmp + beta * C[threadRow * N + threadCol];

If you guessed 'actually putting the entries in C' you got it! tmp will now store the final dot product value, since it has consumed all necessary entries in the row and column chunks, that slid along the whole matrix.

Again, since we moved the pointer for C to the exact chunk/block of output entries that are being computed by the current block, we can just use local thread indexing once more.

One limitation we have to consider with this approach, however, is that, even though we are using shared memory, the actual chunks of data are still re-used i.e. if we consider the red chunk in C instead:

![](../../images/GEMM1/smemchunks2.png)

We can see that we will need the same row-wise chunk sliding window from A, but a different column-wise chunk sliding window from B. 

#### Some Statistics
As eager as you are, before we jump into the next kernel and keep leveling up our optimisations, let's talk about performance, quantified by some stats.

All in all, theres a 50% improvement over the previous kernel - quite an improvement, but not higher since the previous kernel had good L1 cache hit rates. Shared memory just gives us a more surefire, programmer-managed way to reuse data.

But, this is mind, this kernel achives approximately 2200 GLPOS, which seems like a lot, until you look at the maximum the GPU can provide being 30 TFLOPS, lots more optimising to be done.

The GPU used by Simon to test out the kernels was a NVIDIA RTX A6000 GPU, which had a maximum of 48KB of shared memory space available for each block. Having a chunk size of 32, we then can see that both the chunks needed (one from A, one from B) uses 2*32*32*4B = 8KB. 

It's good that we are not exceeding our limit, but there are some caveats to talk about here. Each SM also has a maximum amount of SMEM it has available to it, that it can allocate to its resident blocks, for instance the A6000 GPU gives 100KB of SMEM to each SM.

Note that this does not mean that each SM is not constrained to only 2 blocks since each block has 48KB of SMEM, and any further blocks would overcrowd the SM. I initially thought that if we added more blocks, but they all had ununused capacity e.g. we had 4 blocks that all used only 24KB out of the 48KB, this would be restricted by the SM from the get-go, as when more of the 48KB is attempted to be filled up by the blocks, then the SM would not be able to store them. However, how it actually works is that this is all checked at launch time, solely dependent on the kernel.

Firstly, if the kernel requests more shared memory than the max available per block, kernel launch fails. Secondly, if the kernel requests less shared memory than the max available per block, this will the guaranteed, constant amount of SMEM used by all the blocks launched by this kernel, they will not exceed this amount.

This means that if we has max SMEM per SM = 100KB and max SMEM per block = 50KB, but our kernel was defined as requesting only 25KB per block, then we fit up to 4 blocks per SM. This is assuming that the other constraints such as register count and warp count per SM don't interfere.

This leads us to a very important concept in CUDA, occupancy

##### Occupancy
I'll just let the NVIDIA docs take over for a second.

_Occupancy is the ratio of number of active warps per multiprocessor to the maximum number of possible active warps_

So, occupancy ties in to our discourse of factors that limit the number of blocks, and in turn, warps.

The three main limits being Shared memory, Threads and Registers, with the values for registers per thread, SMEM per block and threads per block being defined by the kernel. 

Let's walk through each of these for our actual kernel to get a intuitive sense of what is happening. Don't worry about the other numbers popping out of thin air, these are the hardware stats with respect to the GPU used by Simon, and are obtained from the cudaGetDeviceProperties API.   Again, the SMEM per block, registers per thread and threads per block are the only values that are variable, and controlled by the kernel i.e. the amount of allocated shared memory, the block size defined, etc.

- Shared memory:
    SMEM needed per block = 8192B of SMEM per block + 1024B of CUDA runtime overhead per block (this is a constant value,  and is a bookkeeping costs that is implementation dependant) = 9216B per block
    Number of blocks that can fit in a SM = 102400B of SMEM per SM / 9216B of SMEM per block = 11.11 -> 11 blocks upper limit

- Threads:
    Number of blocks that can fit in a SM = 1024 threads per block / 1536 max threads per SM -> 1 block upper limit

- Registers:
    37 registers per thread * 32 threads per warp = 1184 registers per warp - register allocation granularity is 256 registers on a warp level so we round it up to 1280 registers per warp
    1024 threads / 32 = 32 warps per block and hence 1280 registers per warp * 32 warps per block = 40960 registers per block. since the max registers per Sm = 65536, we have 1 block upper limit

So this kernel is limited by both the number of threads per block and the number of registers per thread. We cannot load more than one block per SM, which gives us a final occupancy of 32 active warps / 48 max active wraps = 66%

This occupancy rate is acceptable, so it doesn't really illuminate to us why we are still quite a ways off from peak GPU performance.

This leads us to explore other sources of information about the kernel, leading to another aside: PTX

### PTX
I will assume the reader is somewhat familiar with low-level languages (if not, read up, they are COOL) and with that, we can very easily define Parallel Thread Eecution (PTX) as being the assembly language of CUDA. 

It is NVIDIA's intermediate ISA that is used for GPUs, generated using the NVCC compile that we mentioned before. It essentially exists between our high-level CUDA C++ code and the final machine code.

As is the power of other IRs, CUDA code being compiled into PTX which can be compiled by the driver into the final machine code specific to the GPU architecture - essentially, makes our CUDA code portable across multiple GPU architectures.

PTX also allows NVIDIA to change hardware details and make optimizations without breaking our code. Since the driver can translate PTX to the latest machine instructions, old kernels are still supported.

Advanced devs can even write inline PTX directly in CUDA, giving them more control than CUDA C.

So we look to PTX to check how our CUDA cope maps to low-level instructions, which may illuminate inefficient code, which can be debugged for performance gains.


Back to our kernel, the inner loop looks like this in PTX

In [ ]:
ld.shared.f32   %f91, [%r8+3456];
ld.shared.f32   %f92, [%r7+108];
fma.rn.f32      %f93, %f92, %f91, %f90;

Which corresponds to:

In [ ]:
tmp += As[threadIdx.y * BLOCKSIZE + dotIdx] *
       Bs[dotIdx * BLOCKSIZE + threadIdx.x];

The assembly code probably rings some bells while also seeming a bit unfamiliar. Diverging from most low-level languages PTX supports loads from different memory stores and in this case *ld.shared* is a load from shared memory. The other instruction *fma* is our fixed multiply add.

Even though the shared memory load with have a way lower latency than global memory load, it is still higher in latency than the simple FMA instruction, which means that the warp scheduler (which executes instructions) often has to stall, waiting for the loads before the FMA can be done. This means that us having a serialized loads then FMA for all 32 iterations, causes the warp to spend a lot of cycles stalled.

This can be contextualized by looking at the warp states :

![](../../images/GEMM1/warpstates.png)

This shows how many cycles were spent in each state, and we can clearly see that the most common state is 'Stall MIO Throttle'. Using the [Kernel Profiling Guide](https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#metrics-reference), which you should definetely keep handy somehwere, we can see that the Stall MIO Throttle description is as follows: _Warp was stalled waiting for the MIO (memory input/output) instruction queue to be not full. This stall reason is high in cases of extreme utilization of the MIO pipelines, which include special math instructions, dynamic branches, as well as shared memory instructions_


So this confirms that threads (might even be a single thread, since even that is enough to make the whole warp stall) are waiting for the shared memory access to return before they can do the arithmetic operation.

How do we improve on this? Find out on our next kernel!